<font size="+4" color=purple><u><center>US Police Fatal Shooting Analysis (2015-2020)</center></u></font>

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Table of content</h3>

* [Introduction](#intro)
* [Data](#data)
* [1. Yearly death report : 2015-2020](#1)
* [2. Quarterly death report for every year](#2)
* [3. Monthly death report for every year](#3)
* [4. Most Death recorded Months](#4)
* [5. Most Death recorded Month period - Start/middle/end of month](#5)
* [6. Most Death recorded Days](#6)
* [7. Gender proportion](#7)
* [8. Most affected Age group](#8)
* [9. Most affected Race](#9)
* [10. Most suffered State](#10)
* [11. Most suffered City](#11)
* [12. Flee or not?](#12)
* [13. Armed or not?](#13)
* [14. Analysis of Black people](#14)
* [15. Analysis of all Race victims in US](#15)
* [16. Mental Illness](#16) 
* [17. Manner of Death](#17) 
* [18. Threat Level](#18) 

<a id="intro"></a>
<font size="+2" color="blue"><b>Introduction</b></font><br>

Let us look at the dataset from various angles and draw insights.

<a id="data"></a>
<font size="+2" color="blue"><b>Data</b></font><br>

This is a lovely dataset to work upon. 



<a id="packages"></a>
<font size="+2" color="blue"><b>Import libraries and packages</b></font><br>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
import operator
import plotly.express as px

from urllib.request import urlopen
import json

import seaborn as sns
from matplotlib import rcParams
import plotly.graph_objects as go
%matplotlib inline
import matplotlib.pyplot as plt

import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as py

<a id="packages"></a>
<font size="+2" color="blue"><b>Preprocessing</b></font><br>

In [ ]:
data_df = pd.read_csv("../input/data-police-shootings/fatal-police-shootings-data.csv",parse_dates = ['date'])
data_df

In [ ]:
# Feature Generation
data_df['month'] = pd.DatetimeIndex(data_df['date']).month
data_df['year'] = pd.DatetimeIndex(data_df['date']).year
data_df['month_name']=data_df['date'].dt.strftime('%B')
data_df['month_num']=data_df['date'].dt.strftime('%m')
data_df['weekdays']=data_df['date'].dt.strftime('%A') 
data_df['year_quarter'] =data_df.date.dt.to_period("Q")
data_df['date_num']=data_df['date'].dt.strftime('%d').astype(int)
data_df['date_categ']=np.where(data_df['date_num']<11,"First Third",np.where((data_df['date_num']>=11) & (data_df['date_num'] < 20),"Second Third",'Last Third'))

data_df['race_name']=np.where(data_df['race']=='W','White',np.where(data_df['race']=='B','Black',
np.where(data_df['race']=='N','Native American',np.where(data_df['race']=='H','Hispanic',
np.where(data_df['race']=='A','Asian',np.where(data_df['race']=='O','Others','Not Specified'))))))
data_df

<a id="1"></a>
<font size="+2" color="blue"><b>Yearly Death numbers from 2015 - 2020 </b></font><br>

In [ ]:
killings_year = data_df[['year']]
killings_year['kills'] = 1
killings_year = killings_year.groupby('year').sum()
killings_year = killings_year.reset_index()
def line_plot(data,var):    
    trace = go.Scatter(x=data[var],y=data['kills'],line=dict(color='darkgreen',width=2))    
    layout = dict(title='Deaths per {}'.format(var))    
    fig = dict(data = [trace], layout=layout)
    py.iplot(fig)
line_plot(killings_year,'year')

<font size="+1" color="maroon">Insights: 
    
* Police shootings have been between 980 and 1000 for the years 2015-2019.
* We have data till June 2020, the number of shootings are already 481, we have already reached half of usual range.
  
</font>

<a id="2"></a>
<font size="+2" color="blue"><b>Quarterly Death numbers from 2015-2020</b></font><br>

In [ ]:
quarterly_df = data_df['date'].groupby(data_df.date.dt.to_period("Q")).agg('count').to_frame(name="count").reset_index()
quarter_year = []
for i in quarterly_df['date']:
    quarter_year.append(str(i))
    
quarterly_df['year'] = quarterly_df['date'].dt.strftime('%Y') 
def plot_quarter(year,color):
    temp_quarter=[]
    for i in quarterly_df.loc[quarterly_df['year']==year]['date']:
        temp_quarter.append(str(i))        
    trace=go.Bar(x=temp_quarter, y = quarterly_df.loc[quarterly_df['year']==year]['count'],
           name=year,marker_color=color)
    return trace

fig = make_subplots(rows=3, cols=2,subplot_titles=("2015", "2016","2017","2018","2019","2020"))
fig.add_trace(plot_quarter('2015','red'),row=1,col=1)
fig.add_trace(plot_quarter('2016','blue'),row=1,col=2)
fig.add_trace(plot_quarter('2017','green'),row=2,col=1)
fig.add_trace(plot_quarter('2018','orange'),row=2,col=2)
fig.add_trace(plot_quarter('2019','indigo'),row=3,col=1)
fig.add_trace(plot_quarter('2020','violet'),row=3,col=2)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title_text='Deaths Every Year - Quarter wise', title_x=0.5,showlegend=False)
fig.show()

<font size="+1" color="maroon">Insights: 
    
* Year 2015 - Quarters - Q3, Q4 are higher than Quarters Q1, Q2
* Year 2016 - Quarters - Q1 is higher than Quarters Q2, Q3, Q4
* Year 2017 - Quarters - Q1 is higher than Quarters Q2, Q3, Q4
* Year 2018 - Quarters - Q1 is higher than Quarters Q2, Q3, Q4
* Year 2019 - Quarters - Q4 is higher than Quarters Q1, Q2, Q3
* Year 2019 - Quarters - Q1 is higher than Quarter Q2
* Other information - social, political during those peak months, quarters will be helpful in forming an opinion or statement.
  
</font>

<a id="3"></a>
<font size="+2" color="blue"><b>Monthly Death numbers for month/year</b></font><br>

In [ ]:
monthly_df=data_df['date'].groupby(data_df.date.dt.to_period("M")).agg('count').to_frame(name="count").reset_index()
month_year=[]
for i in monthly_df['date']:
    month_year.append(str(i))
    
monthly_df['year']=monthly_df['date'].dt.strftime('%Y') 
def plot_month(year,color):
    temp_month=[]
    for i in monthly_df.loc[monthly_df['year']==year]['date']:
        temp_month.append(str(i))        
    trace=go.Bar(x=temp_month, y=monthly_df.loc[monthly_df['year']==year]['count'],
           name=year,marker_color=color)
    return trace

fig = make_subplots(rows=3, cols=2,subplot_titles=("2015", "2016","2017","2018","2019","2020"))
fig.add_trace(plot_month('2015','red'),row=1,col=1)
fig.add_trace(plot_month('2016','blue'),row=1,col=2)
fig.add_trace(plot_month('2017','green'),row=2,col=1)
fig.add_trace(plot_month('2018','orange'),row=2,col=2)
fig.add_trace(plot_month('2019','indigo'),row=3,col=1)
fig.add_trace(plot_month('2020','violet'),row=3,col=2)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title_text='Deaths Every Year', title_x=0.5,showlegend=False)
fig.show()

<font size="+1" color="maroon">Insights: 
    
* Year ending months like Nov, Dec the violence is low.
* There is police violence spurt in Jun/Jul months
* May 2020 has been exception, the police killings have been at 110 can be attributed to George Floyd killing.
* March 2018 the police killings have been at 109.
* Dec 2019 has been exception (december being peaceful months), the police killings have been at 106. 
  
</font>

<a id="4"></a>
<font size="+2" color="blue"><b>Monthly Death numbers year on year</b></font><br>

In [ ]:
monthly_shootouts = data_df[['year','month']]
monthly_shootouts['kills'] = 1
fig = px.bar(monthly_shootouts, x='month', y='kills',color_discrete_sequence =['magenta'], facet_col='year')
fig.show()

<a id="5"></a>
<font size="+2" color="blue"><b>Monthly Death numbers - which portion of the month ? </b></font><br>

* Here, we divide the month into first 10 days, second 10 days and last 10 days. 
* Usually last 10 days is known to be stressful to meet deadlines of our work. 
* First ten days is known to be stressful to pay bills or new expenses are encountered.

In [ ]:
only_month=data_df.groupby(['month_name','date_categ'])['id'].agg('count').reset_index().rename(columns={'id':'count'})
only_month['month_name'] = pd.Categorical(only_month['month_name'],categories=['January','February','March','April','May','June','July','August','September','October','November','December'],ordered=True)
only_month = only_month.sort_values('month_name')

fig = go.Figure(data=[
    go.Bar(name='First Third', x=only_month[only_month['date_categ']=='First Third']['month_name'], y=only_month[only_month['date_categ']=='First Third']['count']),
    go.Bar(name='Second Third', x=only_month[only_month['date_categ']=='Second Third']['month_name'], y=only_month[only_month['date_categ']=='Second Third']['count']),
    go.Bar(name='Last Third', x=only_month[only_month['date_categ']=='Last Third']['month_name'], y=only_month[only_month['date_categ']=='Last Third']['count'])
])

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title_text='Deaths - All Months',xaxis_title='Months', yaxis_title='Total number of kills', title_x=0.5,barmode='stack')
fig.show()

<font size="+1" color="maroon">Insights: 
    
* Last 10 days of the month have been most violence prone
* First 10 days of the month have also seen most shootings
* Peacful period of the month is the middle 10 days
  
</font>

<a id="6"></a>
<font size="+2" color="blue"><b>Week days numbers - which days do shootings occur ? </b></font><br>

We have got the day number and we look at if more shootings occur on any particular day or a weekday.

In [ ]:
weekdays_df=data_df['weekdays'].value_counts().reset_index().rename(columns={'index':'weekdays','weekdays':'count'})
weekdays_df['weekdays'] = pd.Categorical(weekdays_df['weekdays'],categories=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],ordered=True)
weekdays_df = weekdays_df.sort_values('weekdays')


fig = ff.create_table(weekdays_df, height_constant=60)
fig.add_trace(go.Scatter(
    x= weekdays_df['weekdays'], 
    y= weekdays_df['count'], 
    xaxis='x2', yaxis='y2',
    mode="markers",marker_size=12
))

fig.update_layout(
    title_text = 'Weekdays Death Report',
    title_x=0.5,
    margin = {'t':50, 'b':100},
    xaxis = {'domain': [0, .5]},
    xaxis2 = {'domain': [0.6, 1.]},
    yaxis2 = {'anchor': 'x2', 'title': 'Count'}
)

fig.show()

<font size="+1" color="maroon">Insights: 
    
* Weekdays like Wednesday, Tuesday and Thursday have seen more killings.
* Saturday, Monday have least shootings.
* Weekends are more peaceful than the weekdays.
  
</font>

<a id="7"></a>[](http://)
<font size="+2" color="blue"><b>Analysis of Gender </b></font><br>
Is there any particular gender targetted ?

In [ ]:
shoot_gender=data_df.groupby(['year','gender']).agg('count')['id'].to_frame(name='count').reset_index()
shoot_gender_male=shoot_gender.loc[shoot_gender['gender']=='M']
shoot_gender_female=shoot_gender.loc[shoot_gender['gender']=='F']

male=go.Bar(x=shoot_gender_male['year'],y=shoot_gender_male['count'],marker=dict(color='brown'),name="male")
female=go.Bar(x=shoot_gender_female['year'],y=shoot_gender_female['count'],marker=dict(color='orange'),name="female")
data=[male,female]

fig = go.Figure(data)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title="Death Toll - Gender",title_x=0.5,xaxis=dict(title="Year"),yaxis=dict(title="Number of Victims"),
                   barmode="group")
fig.show()

<font size="+1" color="maroon">Insights: 
    
* Number of male victims are around 900 every year
* Number of female victims are around 40-50 every year.
* Female victims are far lesser than male victims.
  
</font>

<a id="8"></a>
<font size="+2" color="blue"><b>Analysis of Age </b></font><br>
Is there any particular age group that is more prone to shootings ?

In [ ]:
ag_bool = (data_df['age'] > 60 )
data_df[ag_bool].shape[0]

In [ ]:
hist_data = [data_df['age'].dropna()]
group_labels = ['Age'] 
colors=["brown"]
fig=go.Figure()
fig=ff.create_distplot(hist_data, group_labels,bin_size=10,colors=colors)
fig.update_layout(title_text="Distribution of Age",title_x=0.5)
fig.show()

<font size="+1" color="maroon">Insights: 
* There are around 4660 victims between 18-60 years.  
* Number of victims less than thirteen years is  3.
* Number of victims between 13-17 years is 94.
* Number of victims more than sixty years is 294.
  
</font>

In [ ]:
age_df=data_df.groupby(['year','age_freq']).agg('count')['id'].to_frame(name='count').reset_index()
age_df

In [ ]:
data_df['age_freq']=np.where((data_df['age'] < 13),'<13',np.where((data_df['age'] > 12) & (data_df['age'] < 18) ,'13-17',np.where(data_df['age']<18,'<18',np.where((data_df['age']>17)&(data_df['age']<=30),'18-30',
np.where((data_df['age']>30)&(data_df['age']<=50),'31-50',np.where(data_df['age']>70,'70+',
np.where((data_df['age']>50)&(data_df['age']<=70),'51-70',"Not Specified")))))))

age_df=data_df.groupby(['year','age_freq']).agg('count')['id'].to_frame(name='count').reset_index()

fig = go.Figure()
fig.add_trace(go.Box(y=age_df.loc[(age_df['age_freq']=="<13")]['count'], name='<13 (Pre-Teen)', marker_color = 'brown',boxmean=True))
fig.add_trace(go.Box(y=age_df.loc[(age_df['age_freq']=="13-17")]['count'], name = '13-17 (Teen below 18)', marker_color = 'grey',boxmean=True ))
fig.add_trace(go.Box(y=age_df.loc[(age_df['age_freq']=="18-30")]['count'], name = '18-30 (Youth)', marker_color = 'green',boxmean=True ))
fig.add_trace(go.Box(y=age_df.loc[(age_df['age_freq']=="31-50")]['count'], name = '31-50 (Middle age)',marker_color = 'red',boxmean=True ))
fig.add_trace(go.Box(y=age_df.loc[(age_df['age_freq']=="51-70")]['count'], name = '51-70 (Above Middle age) ', marker_color = 'orange',boxmean=True ))
fig.add_trace(go.Box(y=age_df.loc[(age_df['age_freq']=="70+")]['count'], name = '70+ (Senior Citizens)', marker_color = 'violet',boxmean=True ))
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title="Death Toll - All Ages",title_x=0.5,xaxis=dict(title="Age"),yaxis=dict(title="Number of Victims"))

fig.show()

<font size="+1" color="maroon">Insights of killins over years 2015 to June 2020 - age distribution bins: 
* Over the years underage shooting is not seen - Last 2-3 years there have been no shootings of victims under 13.  
* Median/mean number of victims aged between 13-17 is around 15.
* Victims between ages 31-50  median of 444 killlings over the years can be seen.
* Victims between ages 18-30  median of 315 killlings over the yearscan be seen.
* Victims between ages 51-70  median of 143 killlings over the years can be seen.
* Victims of ages 70+  median of 12 killlings over the years can be seen.  
</font>

In [ ]:
age_df=data_df.groupby(['year','age_freq']).agg('count')['id'].to_frame(name='count').reset_index()
age_df

<a id="9"></a>
<font size="+2" color="blue"><b>Analysis of Race </b></font><br>
Is there any particular racial group that is more prone to shootings ?

In [ ]:
PLOT_BGCOLOR='#DADEE3'

race_counts = dict(data_df.race_name.value_counts())
colors = ['#ED1C22','#FEC907','#BF3D09','#7CBB15', '#92D0FF', '#50ADE5','#1373C7','#AD309C','#3BD5C9']
fig = go.Figure(data=[go.Pie(labels=list(race_counts.keys()),values=list(race_counts.values()))])
fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=1.5)))
fig.update_layout(title='Races of people shot',width=700,height=400,barmode='stack',template='seaborn',
                 paper_bgcolor=PLOT_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR,hovermode='x unified',
                 legend=dict(title='<b><i>Type of Races</i></b>',x=0.835,bgcolor=PLOT_BGCOLOR),margin=dict(t=35,b=10,l=10,r=10),
                 xaxis=dict(title='Number of Shootings',mirror=True,linewidth=2,linecolor='black',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=2,linecolor='black',gridcolor='darkgray'))
fig.show()

<font size="+1" color="maroon">Insights: 
* Whites have been most common race of victims who were shot down (45% of total shootings).  
* Blacks numbers have been little above half of white's (24% of total shootings).
* Hispanic victims closelyfollow blacks (at 16%)
</font>

In [ ]:
shoot_race=data_df.groupby(['year','month_num','race_name']).agg('count')['id'].to_frame(name='count').reset_index()
shoot_race['monthly']=shoot_race['year'].astype(str)+"-"+shoot_race['month_num'].astype(str)

def plot_month_race(race,color):
    temp_month=[]
    for i in shoot_race.loc[(shoot_race['race_name']==race)]['monthly']:
        temp_month.append(str(i))
    trace=go.Bar(x=temp_month, y= shoot_race.loc[(shoot_race['race_name']==race)]['count'],
           name=race,marker_color=color)
    return trace

fig = make_subplots(rows=3, cols=2,subplot_titles=("Black", "White","Hispanic","Asian","Native American","Others"))

fig.add_trace(plot_month_race('Black','brown'),row=1,col=1)
fig.add_trace(plot_month_race('White','deepskyblue'),row=1,col=2)
fig.add_trace(plot_month_race('Hispanic','green'),row=2,col=1)
fig.add_trace(plot_month_race('Asian','red'),row=2,col=2)
fig.add_trace(plot_month_race('Native American','orange'),row=3,col=1)
fig.add_trace(plot_month_race('Others','violet'),row=3,col=2)

fig.update_layout(title_text='Deaths - All Race',title_x=0.5)
fig.show()

<font size="+1" color="maroon">Insights: 
* All the years - Whites, Blacks, Hispanics victims were killed in all the months of all the years 
* Asian and Native Americans were not killed in some months of some years

</font>

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=shoot_race.loc[(shoot_race['race_name']=="Black")]['count'], name='Black',
                marker_color = 'brown',boxmean=True))
fig.add_trace(go.Box(y=shoot_race.loc[(shoot_race['race_name']=="White")]['count'], name = 'White',
                marker_color = 'grey',boxmean=True ))
fig.add_trace(go.Box(y=shoot_race.loc[(shoot_race['race_name']=="Hispanic")]['count'], name = 'Hispanic',
                marker_color = 'green',boxmean=True ))
fig.add_trace(go.Box(y=shoot_race.loc[(shoot_race['race_name']=="Asian")]['count'], name = 'Asian',
                marker_color = 'red',boxmean=True ))
fig.add_trace(go.Box(y=shoot_race.loc[(shoot_race['race_name']=="Native American")]['count'], name = 'Native American',
                marker_color = 'orange',boxmean=True ))
fig.add_trace(go.Box(y=shoot_race.loc[(shoot_race['race_name']=="Others")]['count'], name = 'Others',
                marker_color = 'violet',boxmean=True ))
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title="Death Toll - All Races",title_x=0.5,xaxis=dict(title="Race"),yaxis=dict(title="Number of Victims"))

fig.show()

<a id="10"></a>
<font size="+2" color="blue"><b>Which State has seen more shootings? </b></font><br>
Is there any particular state or states that is more prone to shootings ?

In [ ]:
shoot_state=data_df['state'].value_counts().to_frame().reset_index().rename(columns={'index':'state','state':'count'}).sort_values(by='count',ascending=False)
fig = go.Figure(go.Bar(
    y= shoot_state['state'].sort_index(ascending=False), 
    x= shoot_state['count'].sort_index(ascending=False),  
     orientation='h',
      text=shoot_state['count'].sort_index(ascending=False),
    textposition='outside',
    marker_color=shoot_state['count'].sort_index(ascending=False),
))
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title_text='Fatal Killing - All States',yaxis_title='States',
                 xaxis_title='Total number of victims', title_x=0.5,height=1000)
fig.show()

* <font size="+1" color="maroon">Insights: 
* Populous states such as CA, TX, FL have seen shootings 
* Shootings in CA is 789, while TX is 481 (little more than half of CA shootings)
* States like VT, RI are below 10.
</font>

<a id="11"></a>
<font size="+2" color="blue"><b>Which City has seen more Shootings? </b></font><br>
Is there any particular city or cities that is more prone to shootings ?

In [ ]:
shoot_city=data_df['city'].value_counts().to_frame().reset_index().rename(columns={'index':'city','city':'count'}).sort_values(by='count',ascending=False)

fig = go.Figure(go.Bar(
    x= shoot_city['city'][:20], 
    y= shoot_city['count'][:20],  
    text=shoot_state['count'][:20],
    textposition='outside',
    marker_color=shoot_state['count'][:20]
))

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title_text='Fatal Killing - Top 20 Cities',xaxis_title='Cities', yaxis_title='Total number of kills', title_x=0.5)
fig.show()

<font size="+1" color="maroon">Insights: 
* Busy cities such as Los Angeles, Phoenix, Houston have seen shootings 
* Shootings in LA is 799, while Phoenix is 481 (little more than half of LA shootings)

</font>

<a id="12"></a>
<font size="+2" color="blue"><b>Fleeing or not Fleeing? </b></font><br>
Are there any instances where the suspect was not fleeing but shot ?

In [ ]:
flee_counts = dict(data_df.flee.value_counts())
fig = go.Figure(data=[go.Pie(labels=list(flee_counts.keys()),values=list(flee_counts.values()))])
fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=1.5)))
fig.update_layout(title='Fleeing ways',width=600,height=600,barmode='stack',template='seaborn',
                 paper_bgcolor=PLOT_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR,hovermode='x unified',
                 legend=dict(title='<b><i>Ways of Flee</i></b>',x=0.835,bgcolor=PLOT_BGCOLOR),margin=dict(t=35,b=10,l=10,r=10),
                 xaxis=dict(title='Number',mirror=True,linewidth=2,linecolor='black',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=2,linecolor='black',gridcolor='darkgray'))
fig.show()

<font size="+1" color="maroon">Insights : 
* 66% of the victims were not fleeing, so why were they shot ? 
</font>

<a id="13"></a>
<font size="+2" color="blue"><b>Armed or not Armed? </b></font><br>
Are there any instances where the suspect was not armed but shot ?

In [ ]:
armed=list(data_df['armed'].dropna().unique())
fig, (ax2) = plt.subplots(1,1,figsize=[17, 10])
wordcloud2 = WordCloud(background_color='black',colormap="OrRd_r", 
                        width=600,height=400).generate(" ".join(armed))
ax2.imshow(wordcloud2,interpolation='bilinear')
ax2.axis('off')
ax2.set_title('Most Used Arms',fontsize=35)

In [ ]:
armed_counts = dict(data_df.armed.value_counts())
fig = go.Figure(data=[go.Pie(labels=list(armed_counts.keys()),values=list(armed_counts.values()))])
fig.update_traces(hoverinfo='label+value', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=1.5)))
fig.update_layout(title='Arms used',width=600,height=600,barmode='stack',template='seaborn',
                 paper_bgcolor=PLOT_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR,hovermode='x unified',
                 legend=dict(title='<b><i>Type of Arms</i></b>',x=0.835,bgcolor=PLOT_BGCOLOR),margin=dict(t=35,b=10,l=10,r=10),
                 xaxis=dict(title='Number of Shootings',mirror=True,linewidth=2,linecolor='black',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=2,linecolor='black',gridcolor='darkgray'))
fig.show()

<font size="+1" color="maroon">Insights : 
* 59% of the victims had guns. 
* As per word cloud, gun, knife, metal, vehicle were most used.

</font>

<a id="14"></a>
<font size="+2" color="blue"><b>Snapshot of Black people killed</b></font><br>


In [ ]:
black_state=data_df[data_df['race']=='B']['state'].value_counts().to_frame().reset_index().rename(columns={'index':'state','state':'count'})
black_year=data_df[data_df['race']=='B']['year'].value_counts().to_frame().reset_index().rename(columns={'index':'year','year':'count'})

fig = make_subplots(
    rows=2, cols=2, subplot_titles=("Black People victims in each state", "Yearly Black Victims", "Distribution of Black people by Age",),
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[[{"type": "Choropleth", "rowspan": 2}, {"type": "bar"}],
           [            None                    , {"type": "histogram"}]])

fig.add_trace(go.Choropleth(
    locations=black_state['state'],
    z=black_state['count'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=black_state['state'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Millions USD",showscale = False,),row=1, col=1)

fig.add_trace(go.Bar(
    x= black_year['year'], 
    y= black_year['count'], 
    text=black_year['count'],
            textposition='inside',
    marker_color="chocolate",
    name="Death by year"
),row=1, col=2)

fig.add_trace(go.Histogram(x=data_df[data_df['race']=='B']["age"],nbinsx=10,marker_color='brown',name="Age Distribution",xbins=dict(size=10),
    opacity=1),row=2, col=2)

fig.update_layout(
    title_text='US Police Killing Black People (2015-2020)',
    title_x=0.5,
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.update_layout(template="plotly_dark",showlegend=True)
fig.show()

<a id="15"></a>
<font size="+2" color="blue"><b>Snapshot of all people killed</b></font><br>


In [ ]:
only_race = data_df[data_df['race_name'] != "Not Specified"]['race_name'].value_counts().to_frame().reset_index().rename(columns={'index':'race_name','race_name':'count'})
year_shoot = data_df['year'].value_counts().to_frame().reset_index().rename(columns={'index':'year','year':'count'}).sort_values(by="year")

fig = make_subplots(
    rows = 2, cols = 2, subplot_titles=("Victims in all states", "Victims by Year", "Victims by Race",),
    column_widths = [0.6, 0.4],
    row_heights = [0.4, 0.6],
    specs = [[{"type": "Choropleth", "rowspan": 2}, {"type": "bar"}],
           [            None                    , {"type": "pie"}]])

fig.add_trace(go.Choropleth(
    locations = shoot_state['state'],
    z = shoot_state['count'].astype(float),
    locationmode = 'USA-states',
    colorscale = 'Reds',
    autocolorscale = False,
    text = shoot_state['state'], 
    marker_line_color = 'white', 
    colorbar_title = "Millions USD",showscale = False,),row=1, col=1)

fig.add_trace(go.Bar(
    x = year_shoot['year'], y = year_shoot['count'], 
    text = year_shoot['count'],
    textposition = 'inside', marker_color = "chocolate", name = "Death by year"),row = 1, col = 2)

fig.add_trace(go.Pie(labels=only_race['race_name'], values=only_race['count'],textinfo='percent', insidetextorientation='radial'),row=2, col=2)


fig.update_layout(
    title_text='US Police Killing All Race (2015-2020)', title_x=0.5,
    geo = dict(scope='usa', projection=go.layout.geo.Projection(type = 'albers usa'), showlakes=True, lakecolor='rgb(255, 255, 255)'), )

fig.update_layout(template="plotly_dark")
fig.show()

<a id="16"></a>
<font size="+2" color="blue"><b>Analysis of  suspects with 'Mental Illness'</b></font><br>


In [ ]:
PAPER_BGCOLOR = '#CDBAB7'
illness_dict = dict(data_df.signs_of_mental_illness.value_counts(dropna=True))
total = sum(illness_dict.values())
fig = go.Figure()
colors = {False:'#FEE803',True:'#AAEEE7'}
annotations=[]
space = 0
for key, value in illness_dict.items():
    fig.add_trace(go.Bar(name=key,x=[value],y=['Mental Illness Count'],orientation='h',
                        marker_line_color='black',marker_line_width=1.5,marker_color=colors[key]))
    annotations.append(dict(xref='x', yref='y',
                            x=space + (value/2), y=0,
                            text=str(int(np.round(value/total,2)*100)) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(0, 0, 0)'),
                                      showarrow=False))
    space+=value
fig.update_layout(barmode='stack',width=1300,height=90,paper_bgcolor=PAPER_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR,
                 hovermode='y',xaxis=dict(mirror=True,linewidth=2,linecolor='black',showgrid=False),
                 yaxis=dict(mirror=True,linewidth=2,linecolor='black',showgrid=False),margin=dict(t=0,b=0,l=0,r=0),
                 legend=dict(title='Mental Illness',y=0.5),annotations=annotations)
fig.show()

<a id="17"></a>
<font size="+2" color="blue"><b>Analysis of 'Manner of Death'</b></font><br>

In [ ]:
state_data = Counter(data_df["manner_of_death"])
sorted_state_data = dict(sorted(state_data.items(), key=operator.itemgetter(1),reverse=True))
fig = px.funnel_area(names = list(sorted_state_data.keys()),values = list(sorted_state_data.values()),title="Manner of death") 
fig.show()

<font size="+1" color="maroon">Insights : 
* Only 5% of victims were tasered. 
</font>

<a id="18"></a>
<font size="+2" color="blue"><b>Analysis of 'Threat Level'</b></font><br>

In [ ]:
rcParams["figure.figsize"] = 6, 6
g = sns.countplot(x = data_df["threat_level"],hue = data_df["gender"])
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3, '{:1}'.format(round(height)), ha="center") 
g.set_title("Threat Level used on gender")

<font size="+1" color="maroon">Insights : 
* Threat levels doesnot help much in determining whether the killing was justified.
</font>

<font size="+3" color="green">Insights : 
I enjoyed writing this NB. Thank you for the dataset. Very relevant and eye opening !!!
</font>

**References**

Big shoutout to Raenish David for NB.
